# Parsing APIs Example

## Intro

Now we will take a look on a real data. When you parse data from web you will often meet API based web-pages. 

For example [zalando.fr](https://www.zalando.fr/accueil-homme/) is API based web-page. 

In this guided lab you will learn how to obtain the links from webpages and extract the data. Read through this doc, execute the cells in order and make sure you understand the explanations. 

*Note: This guided lab uses Google Chrome. Other browsers like Safari and Firefox have similar tools for developers but they work differently. To save your time in following this lab, it is strongly recommended that you install and use Google Chrome.*

## Obtaining the link

Zalando is discount e-store where you can buy clothes and accesories with discount. When we go to the web-page, we can choose different sections. First the general process will be shown using [Children section](https://www.zalando.fr/accueil-enfant/) as example.

Here we will parse data about promotions only. Therefore, final output will be the DataFrame with all the goods under discount.

[![Image from Gyazo](https://i.gyazo.com/fa4874d8e81c7570273bbfb853d66308.png)](https://gyazo.com/fa4874d8e81c7570273bbfb853d66308)


We go to Promos page. Right click of mouse shows us a list of actions possible, from which we select Inspect.

<img src='https://i.gyazo.com/bccbd11d69c9040dc98758d443e32052.png' width="400">


You will see the menu dropdown on the right side or on the bottom of the window. There you should click on Network:


[![Image from Gyazo](https://i.gyazo.com/f7e0db81cbfee67694183d1a7640bf81.png)](https://gyazo.com/f7e0db81cbfee67694183d1a7640bf81)

Right after the developer part will change showing the files behind the page. In order to obtain only useful files we select the following settings:
1. Preserve Log
2. Select XHR files.

[![Image from Gyazo](https://i.gyazo.com/9a899d4441d9d93e795f79747f1e47d5.png)](https://gyazo.com/9a899d4441d9d93e795f79747f1e47d5)

In order to obtain some files we need to scrool down and go forward to second page. 

[![Image from Gyazo](https://i.gyazo.com/0956eb3d5125075a236c9a439c7749c7.png)](https://gyazo.com/0956eb3d5125075a236c9a439c7749c7)

In the Network panel you can see the following files being uploaded. All the data on the web-page is uploaded from the json file, which is one of the following. It is important to understand which file contains what kind of information. 

<a href="https://gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c"><img src="https://i.gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c.png" alt="Image from Gyazo" width="724.8"/></a>

When you find what kind of information you need for the data to be uploaded you just test it. Here we need the article... file:

<a href="https://gyazo.com/78b35bf492994b3f35c0564a21da202a"><img src="https://i.gyazo.com/78b35bf492994b3f35c0564a21da202a.png" alt="Image from Gyazo" width="727.2"/></a>

When we test the link in Chrome inkognito mode we obtain the proper json file:


<a href="https://gyazo.com/b60453fa98454fa29771c731a5174443"><img src="https://i.gyazo.com/b60453fa98454fa29771c731a5174443.png" alt="Image from Gyazo" width="1530.4"/></a>

In order to change the objects in the json file (kind of pagination), you need to change the offset (the number of the first element on the page). in fact, if you take a look on the link, it is easy to unerstand the structure of the link.

# Reading the data

Now the party rocks! When we know how can we obtain the data, it is not a problem to obtain the whole database with all the data from the web-page.
In this lab you will collect your database of Zalando products. You select which goods you want to track. You can define as many filters to your data as you want. Just make sure that the data represents the filters.




In [2]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import urllib.request

In [50]:
url='https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=0&sort=popularity'

#### Collect first 84 object of the of the data (1st page)

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having *family_articles, flags, media* and *sizes* remaining lists (they are exceptions). Hint: use the headers parameter to get the data!

In [8]:
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15","Accept-Language": "en-gb","Accept-Encoding":"br, gzip, deflate","Accept":"test/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Referer":"http://www.google.com/"}

req = requests.get(url, headers=headers)

fp = urllib.request.urlopen(url)
mybytes = fp.read()

response = mybytes.decode("utf8")
fp.close()

In [54]:
results = json.loads(response)

In [55]:
results_flattened = pd.json_normalize(results)
results_flattened

,total_count,sort,articles,query_path,next_page_path,page_gender,premium,filters,total_article_count,plusStatus,...,inCatalogTeaser.content.tracking_data.cf_tracking_params.time_unit,inCatalogTeaser.content.tracking_data.cf_tracking_params.time_unit_count,inCatalogTeaser.content.tracking_data.cf_tracking_params.shop_slot_id,inCatalogTeaser.content.tracking_data.cf_tracking_params.campaign_source_id,inCatalogTeaser.content.tracking_data.cf_tracking_params.request_id,inCatalogTeaser.content.tracking_data.cf_tracking_params.campaign_id,inCatalogTeaser.content.tracking_data.cf_tracking_params.creative_id,inCatalogTeaser.content.click_url,inCatalogTeaser.channel_name,inCatalogTeaser.slot_id
0,28486,popularity,"[{'sku': 'NI114D0C1-Q12', 'name': 'AIR MAX TAI...",/promo-enfant/,/promo-enfant/?p=2,kids,False,"[{'key': 'sizes', 'label': 'Taille', 'url_key'...",28484,non-eligible,...,DAY,1,in_catalog,undefined,b1e3a4be-cdf0-465a-baf8-61cd85d89d6c,85ee0202-3a51-463b-a231-7352e27efe67,88f22699-8309-496f-a9f9-07c1b546bc59,https://www.zalando.fr/collections/yPgnX9M1TQi...,CF,cb224cac-736d-4e00-b757-d8a8007ef951


In [57]:
results_flattened_articles = pd.json_normalize(results_flattened.articles[0])
results_flattened_articles

,sku,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,delivery_promises,price.original,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,amount
0,NI114D0C1-Q12,AIR MAX TAILWIND IV - Baskets basses - black,"[36, 36.5, 37.5, 38, 38.5, 39, 40]",nike-sportswear-air-max-tailwind-baskets-basse...,[{'path': 'NI/11/4D/0C/1Q/12/NI114D0C1-Q12@10....,Nike Sportswear,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -70...",shoe,[],"139,95 €","41,95 €",True,False,True,False,NaN
1,NI113D09V-Q12,AIR MAX 270 EXTREME - Mocassins - black,"[27.5, 28, 28.5, 29.5, 30, 31, 31.5, 33.5]",nike-sportswear-air-max-270-extreme-baskets-ba...,[{'path': 'NI/11/3D/09/VQ/12/NI113D09V-Q12@16....,Nike Sportswear,False,[],"[{'key': 'discountRate', 'value': '-50%', 'tra...",shoe,[],"89,95 €","44,95 €",False,False,False,False,NaN
2,AD116D007-A11,STAN SMITH - Baskets basses - white,"[28, 29, 30, 31, 32, 33, 34, 35]",adidas-originals-stan-smith-baskets-basses-bla...,[{'path': 'AD/11/6D/00/7A/11/AD116D007-A11@12....,adidas Originals,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -40...",shoe,[],"54,95 €","32,95 €",True,False,True,False,NaN
3,1FI13D003-D11,DISRUPTOR KIDS - Baskets basses - silver,"[28, 29, 31, 32, 33, 34, 35]",fila-disruptor-kids-baskets-basses-silver-1fi1...,[{'path': '1F/I1/3D/00/3D/11/1FI13D003-D11@8.j...,Fila,False,[],"[{'key': 'discountRate', 'value': '-50%', 'tra...",shoe,[],"79,95 €","39,95 €",False,False,False,False,NaN
4,TO126K00C-G11,FLAG CREW - Sweatshirt - red,"[12a, 14a, 16a]",tommy-hilfiger-flag-crew-sweatshirt-to126k00c-g11,[{'path': 'TO/12/6K/00/CG/11/TO126K00C-G11@7.j...,Tommy Hilfiger,False,[],"[{'key': 'discountRate', 'value': '-70%', 'tra...",clothing,[],"84,95 €","25,45 €",False,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,GE113G039-D11,AGASIM GIRL - Chaussures premiers pas - silver...,"[23, 24, 25, 26, 27]",geox-agasim-girl-sandales-silverlilac-ge113g03...,[{'path': 'GE/11/3G/03/9D/11/GE113G039-D11@8.j...,Geox,False,[],"[{'key': 'discountRate', 'value': '-70%', 'tra...",shoe,[],"48,95 €","14,85 €",True,False,True,False,NaN
80,GE113D06B-K11,CIAK GIRL FROZEN ELSA - Baskets basses - light...,"[24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 3...",geox-jr-ciak-girl-frozen-baskets-basses-light-...,[{'path': 'GE/11/3D/06/BK/11/GE113D06B-K11@10....,Geox,False,[],"[{'key': 'discountRate', 'value': '-60%', 'tra...",shoe,[],"59,95 €","23,95 €",True,False,True,False,NaN
81,LE226K00O-G11,Sweat à capuche - red,"[4a, 6a, 8a, 12a, 14a, 16a]",levis-veste-polaire-red-le226k00o-g11,[{'path': 'LE/22/6K/00/OG/11/LE226K00O-G11@2.1...,Levi's®,False,[],"[{'key': 'discountRate', 'value': '-55%', 'tra...",clothing,[],"54,95 €","24,65 €",True,False,True,False,NaN
82,GE114D075-C11,BOY - Baskets basses - grey/lime,"[29, 30, 31, 32, 33, 34, 35, 36, 37]",geox-boy-baskets-basses-ge114d075-c11,[{'path': 'GE/11/4D/07/5C/11/GE114D075-C11@12....,Geox,False,[],"[{'key': 'discountRate', 'value': '-60%', 'tra...",shoe,[],"64,95 €","25,95 €",True,False,True,False,NaN


#### Collect all the objects from selected filters. Total number of pages can be found in the same json. Use *sku* column as index.

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having family_articles, flags, media and sizes remaining lists (they are exceptions).

In [58]:
page_count= results["pagination"]["page_count"]
print(page_count)

340


#### Display the trending brand in DataFrame

In [9]:
#I only set the page_count to 2 because otherwises with page_count 858 I got an IncompleteRead error

dataframe = pd.DataFrame()
page_count2=2
for page in range(page_count2):
    url="https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset="+str(page*84)+"&sort=popularity"
    response = requests.get(url, headers=headers)
    
    fp = urllib.request.urlopen(url)
    mybytes = fp.read()
    response = mybytes.decode("utf-8")
    fp.close()
    
    
    results = json.loads(response)
    results_flattened = pd.json_normalize(results)
    results_flattened_articles = pd.json_normalize(results_flattened.articles[0])
    articles = results_flattened_articles.set_index("sku")
    dataframe = dataframe.append(articles)

In [7]:
dataframe_brandname = pd.DataFrame(dataframe["brand_name"].value_counts())
dataframe_brandname.head()

,brand_name
Nike Sportswear,34
Geox,12
Levi's®,10
adidas Originals,9
Jordan,7


#### Display the brand with maximal total discount (sum of discounts on all goods)

In [10]:
dataframe.columns

Index(['name', 'sizes', 'url_key', 'media', 'brand_name', 'is_premium',
       'family_articles', 'flags', 'product_group', 'delivery_promises',
       'price.original', 'price.promotional', 'price.has_different_prices',
       'price.has_different_original_prices',
       'price.has_different_promotional_prices',
       'price.has_discount_on_selected_sizes_only', 'amount'],
      dtype='object')

In [15]:
def clean_currency(x):

    if isinstance(x, str):
        return(x.replace('€', '').replace(',', '.'))
        return(x)
    
dataframe['price_original'] = dataframe['price.original'].apply(clean_currency).astype('float')
dataframe['price_promotion'] = dataframe['price.promotional'].apply(clean_currency).astype('float')
    
dataframe['discount']=dataframe['price_original'] - dataframe['price_promotion']
    
discounts = dataframe.groupby("brand_name")["discount"].sum()
discounts.sort_values(ascending=False)

brand_name
Nike Sportswear            1108.60
Geox                        486.60
Levi's®                     208.00
adidas Originals            175.20
Kaporal                     170.20
Fila                        154.60
Tommy Hilfiger              130.50
Superfit                    108.50
Jordan                      106.90
Cars Jeans                  103.50
D-XEL                       103.25
Desigual                     91.00
Kickers                      91.00
adidas Performance           76.00
Friboo                       67.50
Re-Gen                       66.00
Vans                         63.00
Puma                         57.05
Nike Performance             53.60
Converse                     52.00
Diesel                       49.00
Petit Bateau                 45.50
Benetton                     44.30
Skechers                     38.80
Quiksilver                   38.56
Jack Wolfskin                38.50
Lacoste Sport                37.00
KARL LAGERFELD               35.00
Tiffosi  

#### Display the brands without discount at all

In [28]:
discounts = pd.DataFrame(discounts).reset_index()

In [32]:
discounts['discount']==0

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
50    False
Name: discount, dtype: bool